In [1]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-16 23:13:00--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucced77d48fe67b5709925c5b161.dl-eu.dropboxusercontent.com/cd/0/inline/CzUdrotosFbypmxXwU4Xd5ysL83VAdo1W3DPGOzFsd9HgSb97ac4cc_F-VSx_OrJrL78LCeNYYFLOZbWPdGKTJz_TRcD4Pep1jMid5XocBwWlXBDN5JCu3R-ii_E6B84PBAbl63Nrzwy6qAFD6V27B2T/file?dl=1# [following]
--2025-10-16 23:13:02--  https://ucced77d48fe67b5709925c5b161.dl-eu.dropboxusercontent.com/cd/0/inline/CzUdrotosFbypmxXwU4Xd5ysL83VAdo1W3DPGOzFsd9HgSb97ac4cc_F-VSx_OrJrL78LCeNYYFLOZbWPdGKTJz_TRcD4Pep1jMid5XocBwWlXBDN5JCu3R-ii_E6B84PBAbl63Nrzwy6qAFD6V27B2T/file?dl=1
Resolving ucced77d48fe67b5709925c5b161.dl-eu.dropboxusercontent.com (ucced77d48fe67b5709925c5b161.dl-eu.dro

In [2]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-full/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-full/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-10k/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-10k/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
!pip install datasets
import datasets

In [4]:
!pip install gensim

In [5]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


In [6]:
import nltk #Imports Natural Language Toolkit library
from nltk.corpus import stopwords #Import Stopword module for common words
from gensim.models import Word2Vec #Import Word2Vec
import string #Import String for punctuations characters

#Download stopwords
try:
    stopwords = stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    stopwords = stopwords.words('english')


def preprocess_text(text):
    tokens = text.lower().split() #Tokenize by empty space and convert to lowercase
    tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens] #Remove punctuation
    tokens = [word for word in tokens if word not in stopwords] #Remove stopwords
    return tokens

print("Preprocessing wikitext_small")
processed_small = [preprocess_text(doc['text']) for doc in wikitext_small] #Preprocess the datasets

print("Training Word2Vec model for wikitext_small") #Train Word2Vec models
model_small = Word2Vec(sentences=processed_small, vector_size=50, window=5, min_count=5, epochs=5)
#Vector size 50 = Each word will be represented by a vector of 50 numbers
#Window 5 = Model considers up to 5 words before and 5 words after the target word for context
#Min_count 5 = Filter out rare words by ignoring words that appear less than 5 times, less context to learn meaningful representation
#The model will go through the data 5 times to refine the word vectors


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Preprocessing wikitext_small
Training Word2Vec model for wikitext_small


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Import WordSim csv

import pandas as pd

file_path = '/content/drive/My Drive/wordsim353/combined.csv'

try:
    wordsim_df = pd.read_csv(file_path)
    print("WordSim-353 data loaded successfully.")
    display(wordsim_df.head())
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
    print("Please check the file path in your Google Drive and update the 'file_path' variable.")
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

WordSim-353 data loaded successfully.


,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


In [17]:
from scipy.spatial.distance import cosine
import numpy as np #Import numpy for nan

def get_word_vector(word, model): #Function to get vector for a word, handling words not in vocabulary
    try:
        return model.wv[word] #Gensim's Word2Vec model stores word vectors in the 'wv' attribute
    except KeyError:
        return None #Return None if word is not in the model's vocabulary

computed_similarities = [] #List to store computed cosine similarities

for index, row in wordsim_df.iterrows(): #Iterate through each row in the WordSim353 Data
    #Get words from the WordSim and convert to lowercase
    word1 = row['Word 1'].lower()
    word2 = row['Word 2'].lower()

    #Get vectors for the words from the trained model
    vector1 = get_word_vector(word1, model_small)
    vector2 = get_word_vector(word2, model_small)

    if vector1 is not None and vector2 is not None:
        similarity = 1 - cosine(vector1, vector2)
        #Compute cosine similarity with Cosine() from scipy.spatial.distance computes cosine distance
    else:
        #If either word is not in the model's vocabulary, we cannot compute similarity.
        #Assign NaN (Not a Number) to indicate this.
        similarity = np.nan # Use np.nan for consistency

    computed_similarities.append(similarity)

# Add the computed similarities as a new column to the DataFrame
# The column name indicates which model was used
wordsim_df['Word2Vec_Similarity_Small'] = computed_similarities

print("Cosine similarities computed using model_small for WordSim-353 dataset.")

# Compute and display similarity for additional word pairs
pairs = [
    ('plane', 'car'),
    ('planet', 'sun'),
    ('cup', 'article'),
    ('sugar', 'approach')
]

print("\nCosine similarities for word pairs below :")
for word1, word2 in pairs:
    vector1 = get_word_vector(word1, model_small)
    vector2 = get_word_vector(word2, model_small)

    if vector1 is not None and vector2 is not None:
        similarity = 1 - cosine(vector1, vector2)
        print(f"'{word1}' and '{word2}': {similarity:.4f}")
    else:
        print(f"'{word1}' or '{word2}' not in model vocabulary. Cannot compute similarity.")

Cosine similarities computed using model_small for WordSim-353 dataset.

Cosine similarities for word pairs below :
'plane' and 'car': 0.9496
'planet' and 'sun': 0.9839
'cup' and 'article': 0.6012
'sugar' and 'approach': 0.9768
